## Baseline Submission: Toxic Language Classification 
**w207 Spring 2018 - Final Project Baseline**

**Team: Paul, Walt, Yisang, Joe**



### Project Description 

Our challenge is to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate.  The toxic language data set is sourced from Wikipedia and available as a public kaggle data set. 

Our goal is to use various machine learning techniques used in class to develop high quality ML models and pipelines.  

1. Exercise and build upon concepts covered in class and test out at least 3 kinds of supervised models:
    a. Regression (LASSO, Logistic)
    b. Trees (RF, XGBoost)
    c. DeepLearning (Tensorflow)
2. Using stacking/ensembling methods for improving prediction metrics (K-Means, anomaly detection)
3. Using unsupervised methods for feature engineering/selection

For the baseline proposal, this file contains a first pass run through from data preprocessing to model evaluation using a regression model pipeline. 

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge




### Data Ingestion

In [1]:
import numpy as np
import pandas as pd
import string
import re

#sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

#scipy imports
from scipy.sparse import hstack

#Visualization imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import bokeh
#! pip install bokeh

# target classes
target_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
# read frames localy through csv
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

# Random index generator for splitting training data
# Note: Each rerun of cell will create new splits.
randIndexCut = np.random.rand(len(train_df)) < 0.7

#S plit up data
test_data = test_df["comment_text"]
dev_data, dev_labels = train_df[~randIndexCut]["comment_text"], train_df[~randIndexCut][target_names]
train_data, train_labels = train_df[randIndexCut]["comment_text"], train_df[randIndexCut][target_names]


print('total training observations:', train_df.shape[0])
print('training data shape:', train_data.shape)
print('training label shape:', train_labels.shape)
print('dev label shape:', dev_labels.shape)
print ('labels names:', target_names)

total training observations: 159571
training data shape: (111744,)
training label shape: (111744, 6)
dev label shape: (47827, 6)
labels names: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


### Exploratory Data Analysis

#### Class Imbalance

Let's see how imblanced the label set is in order to have a better understanding with the label quality of the given data set. 

In [10]:
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, output_file, output_notebook

target_counts = train_labels.apply(np.sum,0)
target_counts

output_notebook()


p = figure(x_range=target_names)
p.vbar(x=target_names, top = target_counts, width=0.9)

show(p)

train_labels.head()

Loading BokehJS ...

,toxic,severe_toxic,obscene,threat,insult,identity_hate
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0


In [11]:
total_size = len(train_labels.toxic)

print(sum(train_labels.toxic))
total_normal=len(np.where((train_labels.toxic==0) & (train_labels.severe_toxic==0) & (train_labels.obscene==0) & 
             (train_labels.threat ==0 ) & (train_labels.insult==0) & (train_labels.identity_hate==0))[0])
print("Total normal = %d, total with issues = %d" % (total_normal, total_size-total_normal))
print(target_counts)
      

10720
Total normal = 100373, total with issues = 11371
toxic            10720
severe_toxic      1118
obscene           5955
threat             343
insult            5561
identity_hate     1025
dtype: int64


The data is fairly imbalanced when counting label occurrences. 

Ideas to consider
- Sampling methods
- Custom Cross Validation

### Feature Engineering/Selection (WIP)
....

### Modeling

#### Text Processing

In [12]:
# Basic Count Vectorizer
countVector = CountVectorizer(ngram_range=(1,1))
train_counts = countVector.fit_transform(train_data)

print("Vocabulary size is: {}".format(len(countVector.vocabulary_)))

print("Number of nonzero entries in matrix: {}".format(train_counts.nnz))

#sample column wise sum, we can see that an observation can have multiple classes. 
count_df = pd.DataFrame(train_labels.apply(np.sum,1), columns = ["counts"])
count_df = count_df[((count_df["counts"] >= 1))]
count_df.head(20)

Vocabulary size is: 153456
Number of nonzero entries in matrix: 4867196


,counts
6,4
12,1
16,1
42,4
43,3
44,1
51,2
55,4
56,3
58,2


#### First Pass Logistic Regression

In [13]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver='sag') 
    cv_score = np.mean(cross_val_score(
        classifier, train_counts, train_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('CV score for class {} is {}'.format(name, cv_score))
    classifier.fit(train_counts, train_labels[name])
    
    
print("Mean ROC_AUC: {}").format(np.mean(scores_output))

CV score for class toxic is 0.798116317574296
CV score for class severe_toxic is 0.7765996914864169
CV score for class obscene is 0.7761163880394683
CV score for class threat is 0.617148792692224


KeyboardInterrupt: 

#### Testing on Dev Data

In [6]:
from sklearn.metrics import auc, roc_curve
from sklearn import metrics

dev_Vector = CountVectorizer(ngram_range=(1,1))
dev_counts = countVector.fit_transform(dev_data)

pred_dt = pd.DataFrame()
scores_dev = []
for name in target_names:
    classifier = LogisticRegression(solver='sag') 
    classifier.fit(dev_counts, dev_labels[name])
    scores_dev.append(cv_score)
    output = classifier.predict(dev_counts)
    fpr, tpr, thresholds = metrics.roc_curve(dev_labels[name], output)
    print('Dev score for class {} is {}'.format(name, metrics.auc(fpr,tpr)))
    pred_dt[name] = classifier.predict_proba(dev_counts)[:, 1]
    
    
print("Mean(dev) ROC_AUC: {}").format(np.mean(scores_dev))

Dev score for class toxic is 0.675413012295
Dev score for class severe_toxic is 0.581701390358
Dev score for class obscene is 0.631987934387
Dev score for class threat is 0.503865476473
Dev score for class insult is 0.60758283931
Dev score for class identity_hate is 0.521895703782
Mean(dev) ROC_AUC: 0.676147422434


Score on dev set is worse than training set, thus evidence of overfitting and a need for performance improvement.

The target is multi-label since each observation can be classified as multiple fields.  This is an important distinction from multi-class where each prediction can only be one label.  

## Evaluation

In [7]:
count_df
train_labels["toxic"]

0         0
1         0
2         0
5         0
6         1
9         0
10        0
11        0
13        0
14        0
17        0
18        0
21        0
22        0
24        0
25        0
26        0
27        0
29        0
30        0
31        0
32        0
35        0
36        0
38        0
39        0
41        0
43        1
44        1
46        0
         ..
159535    0
159536    0
159537    0
159538    0
159539    0
159540    0
159541    1
159542    0
159544    0
159545    0
159546    1
159547    0
159549    0
159550    0
159551    0
159554    1
159555    0
159557    0
159558    0
159559    0
159560    0
159561    0
159562    0
159563    0
159564    0
159565    0
159566    0
159567    0
159569    0
159570    0
Name: toxic, dtype: int64

### Submission

In [8]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

prediction_submission = pd.DataFrame()
prediction_submission["id"] = test_df["id"]

# new vector object for all train data for submission
finalTrainVector = CountVectorizer()
finalTrainCount = finalTrainVector.fit_transform(train_df["comment_text"])

# TODO: Using pipelines can clean up repeitive processes
# test set up
#testVector = CountVectorizer()
testCount = finalTrainVector.transform(test_df["comment_text"])

for name in target_names:
    classifier = LogisticRegression(solver='sag') #sag is one kind of solver optimize for multi-label
    clf = classifier.fit(finalTrainCount, train_df["toxic"])
    prediction_submission[name] = clf.predict_proba(testCount)[:, 1]
    #print(prediction_submission)

    
print(prediction_submission.head(10)) # print frame output 
#prediction_submission.to_csv("submission.csv")

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.889472      0.889208  0.889490  0.889300  0.889859   
1  0000247867823ef7  0.250917      0.250663  0.250762  0.250821  0.250742   
2  00013b17ad220c46  0.432613      0.432735  0.432661  0.432668  0.432595   
3  00017563c3f7919a  0.068362      0.068237  0.068175  0.068064  0.068163   
4  00017695ad8997eb  0.424004      0.424707  0.424460  0.424676  0.424557   
5  0001ea8717f6de06  0.340763      0.340854  0.340989  0.340558  0.340962   
6  00024115d4cbde0f  0.124536      0.124778  0.124653  0.124630  0.124505   
7  000247e83dcc1211  0.452367      0.452056  0.452069  0.452164  0.452073   
8  00025358d4737918  0.006227      0.006221  0.006210  0.006212  0.006193   
9  00026d1092fe71cc  0.044093      0.044136  0.044161  0.044094  0.044203   

   identity_hate  
0       0.888301  
1       0.250729  
2       0.432661  
3       0.068241  
4       0.424652  
5       0.340869  
6       0.124839  


The frame contains the output for each class and is saved in a pandas data frame.  

In [14]:
# cell for preprocessing functions
from autocorrect import spell

def find_spelling_errors(textdoc):
    words = textdoc.split()
    words = [ word.lower() for word in words]
    misspelled_words = []
    misspelled_words.append([re.sub(punctuation,'',word) for word in words if re.sub(punctuation,'',word) not in all_words])
    return misspelled_words

def trysplit(word):
    lentmp = len(word)
    return_words=[]
    for i in range(lentmp):
        if all_words.intersection(set([word[:lentmp-i].lower()])):
            return_words.append(word[:lentmp-i])
            if len(word[lentmp-i:]) > 0:
                return_words.extend(trysplit(word[lentmp-i:]))
            else:
                return [word[:lentmp-i]]
            break
    return return_words 

def fix_spelling_errors(textdoc):
    words = textdoc.split()
    return_list = []
    for word in words:
        if all_words.intersection(set([re.sub(punctuation,'',word.lower())])):
            return_list.append(word)
        else:
            # word is not found in the dictionary, try to correct the spelling
            if word == spell(word): # no changes made by the spell checker
                return_list.extend(trysplit(word))
            else:
                return_list.append(spell(word))
    return return_list
    #return [ spell(word.lower()) if re.sub(punctuation,'',word.lower()) not in all_words else word for word in words]
    #return [spell(word) for word in words]

In [15]:
# Calculate a flag for any level of toxix or a unique number for each combination
def bernoulli_toxic_labels (label_vector):
    return [1 if (label_vector['toxic'][x] + label_vector['severe_toxic'][x] +
                label_vector['obscene'][x] + label_vector['threat'][x] +
                label_vector['insult'][x] + label_vector['identity_hate'][x]) > 0 else 0 
            for x in label_vector.index.values]

def binarize_toxic_labels (label_vector):
    return [(label_vector['toxic'][x]*32 + label_vector['severe_toxic'][x]*16 +
                label_vector['obscene'][x]*8 + label_vector['threat'][x]*4 +
                label_vector['insult'][x]*2 + label_vector['identity_hate'][x]) 
            for x in label_vector.index.values]

binary_train_labels = binarize_toxic_labels(train_labels)
binary_dev_labels = binarize_toxic_labels(dev_labels)

bernoulli_train_labels = bernoulli_toxic_labels(train_labels)
bernoulli_dev_labels = bernoulli_toxic_labels(dev_labels)

In [99]:
count_vect_plain = CountVectorizer()
X_train_counts_plain = count_vect_plain.fit_transform(train_data)
X_dev_counts_plain = count_vect_plain.transform(dev_data)

tfidf_vect_plain = TfidfVectorizer()
X_train_tfidf_plain = tfidf_vect_plain.fit_transform(train_data)
X_dev_tfidf_plain = tfidf_vect_plain.transform(dev_data)

count_vect_stop_words = CountVectorizer(stop_words='english')
X_train_counts_stop_words = count_vect_stop_words.fit_transform(train_data)
X_dev_counts_stop_words = count_vect_stop_words.transform(dev_data)

tfidf_vect_stop_words = TfidfVectorizer(stop_words='english')
X_train_tfidf_stop_words = tfidf_vect_stop_words.fit_transform(train_data)
X_dev_tfidf_stop_words = tfidf_vect_stop_words.transform(dev_data)

count_vect_stop_words_max10k = CountVectorizer(stop_words='english', max_features=10000)
X_train_counts_stop_words_max10k = count_vect_stop_words_max1k.fit_transform(train_data)
X_dev_counts_stop_words_max10k = count_vect_stop_words_max1k.transform(dev_data)

tfidf_vect_stop_words_max10k = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_tfidf_stop_words_max10k = tfidf_vect_stop_words_max1k.fit_transform(train_data)
X_dev_tfidf_stop_words_max10k = tfidf_vect_stop_words_max1k.transform(dev_data)

count_vect_stop_words_max5k = CountVectorizer(stop_words='english', max_features=5000,lowercase=False)
X_train_counts_stop_words_max5k = count_vect_stop_words_max5k.fit_transform(train_data)
X_dev_counts_stop_words_max5k = count_vect_stop_words_max5k.transform(dev_data)

tfidf_vect_stop_words_max5k = TfidfVectorizer(stop_words='english', max_features=5000,lowercase=False)
X_train_tfidf_stop_words_max5k = tfidf_vect_stop_words_max5k.fit_transform(train_data)
X_dev_tfidf_stop_words_max5k = tfidf_vect_stop_words_max5k.transform(dev_data)

In [71]:
def score_f1_auc_on_train_dev(dev_vector, train_vector, name):
    multinomial_nb_class = MultinomialNB().fit(train_vector, train_labels[name])
    predicted_labels_dev = multinomial_nb_class.predict(dev_vector)
    fpr, tpr, thresholds = metrics.roc_curve(dev_labels[name], predicted_labels_dev)
    predicted_labels_train = multinomial_nb_class.predict(train_vector)
    fpr1, tpr1, thresholds1 = metrics.roc_curve(train_labels[name], predicted_labels_train)
    f1scoredev = metrics.f1_score(dev_labels[name],predicted_labels_dev,average='micro')
    f1scoretrain = metrics.f1_score(train_labels[name],predicted_labels_train,average='micro')
    aucdev = metrics.auc(fpr,tpr)
    auctrain = metrics.auc(fpr1,tpr1)
    return f1scoredev,aucdev,f1scoretrain,auctrain

In [100]:
scores_all=pd.DataFrame(columns=['set','label','f1dev','aucdev','f1train','auctrain'])

for name in target_names:
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_counts_plain,
                                                   dev_vector=X_dev_counts_plain,name=name)
    scores_all.loc[scores_all.shape[0]] = ['CountPlain',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]

    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_tfidf_plain,
                                                   dev_vector=X_dev_tfidf_plain,name=name)
    scores_all.loc[scores_all.shape[0]] = ['TfidfPlain',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]

    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_counts_stop_words,
                                                   dev_vector=X_dev_counts_stop_words,name=name)
    scores_all.loc[scores_all.shape[0]] = ['CountStopWords',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_tfidf_stop_words,
                                                   dev_vector=X_dev_tfidf_stop_words,name=name)
    scores_all.loc[scores_all.shape[0]] = ['TfidfStopWords',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_counts_stop_words_max10k,
                                                   dev_vector=X_dev_counts_stop_words_max10k,name=name)
    scores_all.loc[scores_all.shape[0]] = ['CountStopWords10k',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_tfidf_stop_words_max10k,
                                                   dev_vector=X_dev_tfidf_stop_words_max10k,name=name)
    scores_all.loc[scores_all.shape[0]] = ['TfidfStopWords10k',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_counts_stop_words_max5k,
                                                   dev_vector=X_dev_counts_stop_words_max5k,name=name)
    scores_all.loc[scores_all.shape[0]] = ['CountStopWords5k',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_tfidf_stop_words_max5k,
                                                   dev_vector=X_dev_tfidf_stop_words_max5k,name=name)
    scores_all.loc[scores_all.shape[0]] = ['TfidfStopWords5k',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]

    # not measuring here for each name
print(scores_all)

                  set          label     f1dev    aucdev   f1train  auctrain
0          CountPlain          toxic  0.947289  0.801548  0.951962  0.845478
1          TfidfPlain          toxic  0.917243  0.567728  0.921266  0.590688
2      CountStopWords          toxic  0.948606  0.788689  0.956096  0.843303
3      TfidfStopWords          toxic  0.920777  0.586300  0.925374  0.612055
4   CountStopWords10k          toxic  0.942334  0.768309  0.945339  0.776567
5   TfidfStopWords10k          toxic  0.944216  0.719495  0.945438  0.725590
6    CountStopWords5k          toxic  0.943944  0.793540  0.946986  0.808163
7    TfidfStopWords5k          toxic  0.943693  0.746089  0.946583  0.761787
8          CountPlain   severe_toxic  0.988500  0.705723  0.987096  0.754863
9          TfidfPlain   severe_toxic  0.990027  0.500000  0.989897  0.499950
10     CountStopWords   severe_toxic  0.989295  0.715463  0.987946  0.765475
11     TfidfStopWords   severe_toxic  0.990027  0.500000  0.989932  0.499968

In [95]:
count_vect_ngram5 = CountVectorizer(ngram_range=(1,10),lowercase=True)
X_train_counts_ngram5 = count_vect_plain.fit_transform(train_data)
X_dev_counts_ngram5 = count_vect_plain.transform(dev_data)

tfidf_vect_ngram5 = TfidfVectorizer(ngram_range=(1,10),lowercase=True)
X_train_tfidf_ngram5 = tfidf_vect_plain.fit_transform(train_data)
X_dev_tfidf_ngram5 = tfidf_vect_plain.transform(dev_data)

In [96]:
for name in target_names:
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_counts_ngram5,
                                                   dev_vector=X_dev_counts_ngram5,name=name)
    scores_all.loc[scores_all.shape[0]] = ['CountNgram5',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
    tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(train_vector=X_train_tfidf_ngram5,
                                                   dev_vector=X_dev_tfidf_ngram5,name=name)
    scores_all.loc[scores_all.shape[0]] = ['TfidfNgram5',name,tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
print(scores_all)

                   set          label     f1dev    aucdev   f1train  auctrain
0           CountPlain          toxic  0.947289  0.801548  0.951962  0.845478
1           TfidfPlain          toxic  0.917243  0.567728  0.921266  0.590688
2       CountStopWords          toxic  0.948606  0.788689  0.956096  0.843303
3       TfidfStopWords          toxic  0.920777  0.586300  0.925374  0.612055
4    CountStopWords10k          toxic  0.942334  0.768309  0.945339  0.776567
5    TfidfStopWords10k          toxic  0.944216  0.719495  0.945438  0.725590
6     CountStopWords5k          toxic  0.946160  0.819693  0.950029  0.833777
7     TfidfStopWords5k          toxic  0.948397  0.747712  0.951362  0.763387
8           CountPlain   severe_toxic  0.988500  0.705723  0.987096  0.754863
9           TfidfPlain   severe_toxic  0.990027  0.500000  0.989897  0.499950
10      CountStopWords   severe_toxic  0.989295  0.715463  0.987946  0.765475
11      TfidfStopWords   severe_toxic  0.990027  0.500000  0.989

In [94]:
scores_all.groupby(['label','set']).max()

f1dev    aucdev   f1train  auctrain
label         set                                                      
identity_hate CountNgram5        0.989629  0.557514  0.987937  0.600515
              CountPlain         0.989629  0.557514  0.987937  0.600515
              CountStopWords     0.990319  0.552641  0.989082  0.633474
              CountStopWords10k  0.985301  0.702826  0.984393  0.724865
              CountStopWords5k   0.989629  0.755866  0.987937  0.813041
              TfidfNgram5        0.992055  0.500000  0.990738  0.499955
              TfidfPlain         0.992055  0.500000  0.990738  0.499955
              TfidfStopWords     0.992055  0.500000  0.990765  0.499968
              TfidfStopWords10k  0.992138  0.565305  0.991337  0.579517
              TfidfStopWords5k   0.992264  0.523600  0.991597  0.556450
insult        CountNgram5        0.964581  0.766856  0.965510  0.813491
              CountPlain         0.964581  0.766856  0.965510  0.813491
              CountStopWords     0.965877  0.756472  0.968240  0.815609
              CountStopWords10k  0.965250  0.785239  0.966871  0.794866
              CountStopWords5k   0.964581  0.828546  0.966414  0.845151
              TfidfNgram5        0.952893  0.514626  0.952660  0.526326
              TfidfPlain         0.952893  0.514626  0.952660  0.526326
              TfidfStopWords     0.953583  0.521545  0.953447  0.534579
              TfidfStopWords10k  0.967320  0.712561  0.967873  0.723822
              TfidfStopWords5k   0.967842  0.720007  0.969851  0.742841
obscene       CountNgram5        0.968762  0.802596  0.967622  0.834807
              CountPlain         0.968762  0.802596  0.967622  0.834807
              CountStopWords     0.970916  0.799564  0.971998  0.846230
              CountStopWords10k  0.973718  0.836659  0.974021  0.837948
              CountStopWords5k   0.969871  0.856984  0.972285  0.874827
              TfidfNgram5        0.952056  0.541055  0.952561  0.556259
              TfidfPlain         0.952056  0.541055  0.952561  0.556259
              TfidfStopWords     0.953625  0.556091  0.954619  0.575570
              TfidfStopWords10k  0.973341  0.768068  0.973278  0.771631
              TfidfStopWords5k   0.972798  0.768918  0.974459  0.787072
severe_toxic  CountNgram5        0.988500  0.705723  0.987096  0.754863
              CountPlain         0.988500  0.705723  0.987096  0.754863
              CountStopWords     0.989295  0.715463  0.987946  0.765475
              CountStopWords10k  0.985218  0.804718  0.984697  0.828027
              CountStopWords5k   0.988500  0.812333  0.987096  0.865368
              TfidfNgram5        0.990027  0.500000  0.989897  0.499950
              TfidfPlain         0.990027  0.500000  0.989897  0.499950
              TfidfStopWords     0.990027  0.500000  0.989932  0.499968
              TfidfStopWords10k  0.990027  0.621406  0.990362  0.647607
              TfidfStopWords5k   0.990382  0.561401  0.990854  0.600486
threat        CountNgram5        0.996341  0.518047  0.995740  0.524108
              CountPlain         0.996341  0.518047  0.995740  0.524108
              CountStopWords     0.996801  0.533050  0.995955  0.548921
              CountStopWords10k  0.990905  0.736914  0.989995  0.772637
              CountStopWords5k   0.996341  0.769115  0.995740  0.835632
              TfidfNgram5        0.997177  0.500000  0.996913  0.502897
              TfidfPlain         0.997177  0.500000  0.996913  0.502897
              TfidfStopWords     0.997177  0.500000  0.996913  0.502897
              TfidfStopWords10k  0.997282  0.518519  0.996957  0.508733
              TfidfStopWords5k   0.997177  0.500000  0.996913  0.502897
toxic         CountNgram5        0.947289  0.801548  0.951962  0.845478
              CountPlain         0.947289  0.801548  0.951962  0.845478
              CountStopWords     0.948606  0.788689  0.956096  0.843303
              CountStopWords10k  0.942334  0.768309  0.945339  0.776567
    